In [11]:
import pandas as pd
import numpy as np

In [41]:
from sklearn.impute import SimpleImputer

data_uni_2 = pd.read_csv("data_uni.csv")

data_uni_2.iloc[:, [24,25,29]]= data_uni_2.iloc[:, [24,25,29]].fillna(0)
column_miss= list(data_uni_2.columns[data_uni_2.isnull().any()])

# IMPUTER ===========================================================================

imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

data_uni_2[column_miss]= imputer.fit_transform(data_uni_2[column_miss])

data_uni_2= data_uni_2.drop(["Course","AWM year 1", "AWM year 2", "AWM year 3", "Overall AWM", "Progression", "Attendance"], axis = 1)

# FORMATING MANUALLY ================================================================

#ethinicty binarisation 
data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("White", "0")
data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("Asian", "1")
data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("Black/Black British African", "1")
data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("Other ethnic background", "1")

# Polar 4 Score Binarisation
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.replace(5, 0)
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.replace(4, 0)
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.replace(3, 0)
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.replace(2, 1)

#splitting X and y ==================================================================
y = data_uni_2["desertion"]
X = data_uni_2.drop(["desertion"], axis = 1)

In [42]:
from pycaret.classification import *
classify = setup(data= data_uni_2, target = "desertion", log_experiment = True, log_plots= True, remove_multicollinearity= True, session_id=1434, multicollinearity_threshold=0.8, log_profile= True, fix_imbalance=True, feature_selection = False, normalize = False,  pca= False, pca_components=0.9, numeric_features=[ "First Sit", "Fails","No Submissions", "Late Submission", "Pass",  "Maths", "UCAS Points"])

,Description,Value
0,session_id,1434
1,Target,desertion
2,Target Type,Binary
3,Label Encoded,"no: 0, yes: 1"
4,Original Data,"(1033, 29)"
5,Missing Values,False
6,Numeric Features,9
7,Categorical Features,19
8,Ordinal Features,False
9,High Cardinality Features,False


In [43]:
compare_models(include=["rf", "ada","gbc"])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9253,0.9453,0.6909,0.8072,0.7411,0.6981,0.7031,0.3860
ada,Ada Boost Classifier,0.9225,0.9334,0.6864,0.8125,0.7257,0.6823,0.6958,0.0670
gbc,Gradient Boosting Classifier,0.9212,0.9488,0.7326,0.7698,0.7406,0.6948,0.7013,0.1160


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=1434, verbose=0,
                       warm_start=False)

In [44]:
rf_2= create_model("rf", cross_validation= True)
rf_2_results = pull()
rf_scores = rf_2_results.loc[["Mean"]]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8630,0.9378,0.4167,0.6250,0.5000,0.4243,0.4360
1,0.9452,0.9788,0.7500,0.9000,0.8182,0.7862,0.7908
2,0.9589,0.9918,0.8333,0.9091,0.8696,0.8452,0.8464
3,0.9583,0.9590,0.8182,0.9000,0.8571,0.8328,0.8341
4,0.9583,0.9940,0.7273,1.0000,0.8421,0.8188,0.8326
5,0.8889,0.8711,0.5455,0.6667,0.6000,0.5362,0.5399
6,0.9167,0.9709,0.6364,0.7778,0.7000,0.6522,0.6566
7,0.9167,0.9061,0.6364,0.7778,0.7000,0.6522,0.6566
8,0.8889,0.8674,0.7273,0.6154,0.6667,0.6006,0.6036


In [46]:
rf_test= create_model("rf", cross_validation= False)
rf_test_results = pull()

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9484,0.9556,0.7556,0.8718,0.8095,0.7798,0.7825


In [49]:
tuned_rf_2= tune_model(rf_2, optimize = "F1")
tuned_rf_2_results = pull()
tuned_rf_scores = tuned_rf_2_results.loc[["Mean"]]


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8904,0.9454,0.5000,0.7500,0.6000,0.5394,0.5543
1,0.9178,0.9604,0.8333,0.7143,0.7692,0.7196,0.7228
2,0.9589,0.9781,1.0000,0.8000,0.8889,0.8641,0.8722
3,0.9306,0.9188,0.8182,0.7500,0.7826,0.7414,0.7424
4,0.9583,0.9866,0.8182,0.9000,0.8571,0.8328,0.8341
5,0.8472,0.8599,0.4545,0.5000,0.4762,0.3870,0.3876
6,0.8889,0.9478,0.5455,0.6667,0.6000,0.5362,0.5399
7,0.9167,0.8972,0.7273,0.7273,0.7273,0.6781,0.6781
8,0.8472,0.8800,0.8182,0.5000,0.6207,0.5319,0.5572


In [32]:
final_rf_2 = calibrate_model(tuned_rf_2)
final_rf_2_results= pull()
final_rf_scores = final_rf_2_results.loc[["Mean"]]



,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9589,0.9883,0.8182,0.9000,0.8571,0.8332,0.8345
1,0.9726,0.9795,0.8333,1.0000,0.9091,0.8931,0.8983
2,0.9589,0.9891,0.9167,0.8462,0.8800,0.8553,0.8563
3,0.9722,0.9985,0.9091,0.9091,0.9091,0.8927,0.8927
4,0.9722,0.9821,0.9091,0.9091,0.9091,0.8927,0.8927
5,0.9861,0.9851,0.9091,1.0000,0.9524,0.9443,0.9457
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,0.9028,0.9791,0.6364,0.7000,0.6667,0.6099,0.6109
8,0.9722,0.9866,0.9091,0.9091,0.9091,0.8927,0.8927


In [19]:
evaluate_model(final_rf_2)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [20]:
gbc_test = create_model("gbc",cross_validation= False)
gbc_test_results= pull()

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9484,0.9753,0.7391,0.8947,0.8095,0.78,0.7847


In [21]:
gbc_2 = create_model("gbc",cross_validation= True)
gbc_2_results= pull()
gbc_scores =gbc_2_results.loc[["Mean"]] 


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9589,1.0000,0.7273,1.0000,0.8421,0.8192,0.8329
1,0.9589,0.9904,0.8333,0.9091,0.8696,0.8452,0.8464
2,0.9726,0.9959,0.9167,0.9167,0.9167,0.9003,0.9003
3,0.9444,0.9881,0.8182,0.8182,0.8182,0.7854,0.7854
4,0.9583,0.9940,0.9091,0.8333,0.8696,0.8448,0.8460
5,0.9861,0.9776,0.9091,1.0000,0.9524,0.9443,0.9457
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,0.9444,0.9806,0.8182,0.8182,0.8182,0.7854,0.7854
8,0.9722,0.9106,0.9091,0.9091,0.9091,0.8927,0.8927


In [22]:
tuned_gbc= tune_model(gbc_2, optimize = "F1")
tuned_gbc_results = pull()
tuned_gbc_scores = tuned_gbc_results.loc[["Mean"]]


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9726,0.9927,0.9091,0.9091,0.9091,0.8930,0.8930
1,0.9726,0.9768,0.8333,1.0000,0.9091,0.8931,0.8983
2,0.9452,0.9918,0.9167,0.7857,0.8462,0.8131,0.8166
3,0.9583,0.9940,0.9091,0.8333,0.8696,0.8448,0.8460
4,0.9583,0.9940,0.9091,0.8333,0.8696,0.8448,0.8460
5,0.9722,0.9762,0.8182,1.0000,0.9000,0.8841,0.8901
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,0.9167,0.9762,0.8182,0.6923,0.7500,0.7004,0.7039
8,0.9861,0.9419,0.9091,1.0000,0.9524,0.9443,0.9457


In [23]:
final_gbc = calibrate_model(tuned_gbc)
final_gbc_results= pull()
final_gbc_scores = final_gbc_results.loc[["Mean"]]


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9589,0.9927,0.8182,0.9000,0.8571,0.8332,0.8345
1,0.9726,0.9822,0.8333,1.0000,0.9091,0.8931,0.8983
2,0.9452,0.9932,0.7500,0.9000,0.8182,0.7862,0.7908
3,0.9722,0.9955,0.9091,0.9091,0.9091,0.8927,0.8927
4,0.9583,0.9940,0.9091,0.8333,0.8696,0.8448,0.8460
5,0.9722,0.9717,0.8182,1.0000,0.9000,0.8841,0.8901
6,0.9861,1.0000,0.9091,1.0000,0.9524,0.9443,0.9457
7,0.9306,0.9762,0.7273,0.8000,0.7619,0.7214,0.7225
8,0.9861,0.9493,0.9091,1.0000,0.9524,0.9443,0.9457


In [24]:
evaluate_model(final_gbc)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [25]:
ada_test= create_model("ada",cross_validation= False)
ada_test_results= pull()

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9613,0.9559,0.8478,0.8864,0.8667,0.844,0.8443


In [26]:
ada= create_model("ada",cross_validation= True)
ada_results= pull()
ada_scores =ada_results.loc[["Mean"]] 


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9589,0.9883,1.0000,0.7857,0.8800,0.8556,0.8647
1,0.9452,0.9795,0.8333,0.8333,0.8333,0.8005,0.8005
2,0.9726,0.9918,1.0000,0.8571,0.9231,0.9065,0.9105
3,0.9583,0.9955,0.9091,0.8333,0.8696,0.8448,0.8460
4,0.9583,0.9925,0.9091,0.8333,0.8696,0.8448,0.8460
5,0.9444,0.9642,0.8182,0.8182,0.8182,0.7854,0.7854
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,0.9028,0.9791,0.9091,0.6250,0.7407,0.6834,0.7016
8,0.9722,0.9344,0.8182,1.0000,0.9000,0.8841,0.8901


In [27]:
tuned_ada= tune_model(ada, optimize = "F1")
tuned_ada_results = pull()
tuned_ada_scores = tuned_ada_results.loc[["Mean"]]


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9452,0.9956,0.9091,0.7692,0.8333,0.8008,0.8048
1,0.9726,0.9781,0.8333,1.0000,0.9091,0.8931,0.8983
2,0.9863,0.9945,1.0000,0.9231,0.9600,0.9518,0.9529
3,0.9583,0.9955,0.9091,0.8333,0.8696,0.8448,0.8460
4,0.9583,0.9925,0.9091,0.8333,0.8696,0.8448,0.8460
5,0.9861,0.9687,0.9091,1.0000,0.9524,0.9443,0.9457
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,0.9444,0.9791,0.9091,0.7692,0.8333,0.8003,0.8043
8,0.9861,0.9434,0.9091,1.0000,0.9524,0.9443,0.9457


In [28]:
final_ada = calibrate_model(tuned_ada)
final_ada_results= pull()
final_ada_scores = final_ada_results.loc[["Mean"]]


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9452,0.9941,0.9091,0.7692,0.8333,0.8008,0.8048
1,0.9726,0.9795,0.8333,1.0000,0.9091,0.8931,0.8983
2,0.9863,0.9932,1.0000,0.9231,0.9600,0.9518,0.9529
3,0.9583,0.9955,0.9091,0.8333,0.8696,0.8448,0.8460
4,0.9583,0.9896,0.9091,0.8333,0.8696,0.8448,0.8460
5,0.9722,0.9627,0.8182,1.0000,0.9000,0.8841,0.8901
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,0.9444,0.9791,0.9091,0.7692,0.8333,0.8003,0.8043
8,0.9861,0.9404,0.9091,1.0000,0.9524,0.9443,0.9457


In [29]:
evaluate_model(final_ada)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [30]:
# results table

resum= pd.concat((ada_test_results,gbc_test_results, rf_test_results, final_ada_scores, final_gbc_scores, final_rf_scores, tuned_ada_scores, tuned_gbc_scores, tuned_rf_scores, ada_scores, rf_scores, gbc_scores), axis =0)
resum

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9613,0.9559,0.8478,0.8864,0.8667,0.8440,0.8443
0,0.9484,0.9753,0.7391,0.8947,0.8095,0.7800,0.7847
0,0.9516,0.9868,0.7174,0.9429,0.8148,0.7876,0.7973
Mean,0.9626,0.9760,0.8742,0.8878,0.8759,0.8541,0.8575
Mean,0.9585,0.9780,0.8129,0.9092,0.8561,0.8321,0.8353
Mean,0.9599,0.9768,0.8386,0.8923,0.8624,0.8391,0.8411
Mean,0.9640,0.9773,0.8833,0.8878,0.8811,0.8601,0.8631
Mean,0.9599,0.9787,0.8568,0.8911,0.8673,0.8439,0.8482
Mean,0.9599,0.9813,0.8303,0.9014,0.8607,0.8376,0.8408
Mean,0.9502,0.9754,0.8652,0.8300,0.8390,0.8101,0.8157
